In [37]:
import sqlite3 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


## Part 1: Wrangling
## Problem 1

In [39]:
conn = sqlite3.connect("lahman2014.sqlite")

query = """
SELECT
    t.teamID,
    t.yearID,
    t.franchID,
    t.W AS wins,
    t.G AS games_played,
    COALESCE(s.total_payroll, 0) AS total_payroll,
    CASE
        WHEN t.G = 0 THEN NULL
        ELSE (t.W * 1.0 / t.G) * 100.0  -- Avoid integer division
    END AS winning_percentage
FROM
    Teams t
LEFT JOIN (
    SELECT
        teamID,
        yearID,
        SUM(salary) AS total_payroll
    FROM
        Salaries
    GROUP BY
        teamID,
        yearID
) s ON t.teamID = s.teamID AND t.yearID = s.yearID
"""

# Execute the query
results = conn.execute(query).fetchall()

# Print the first few rows of the results
for row in results:
    print(row)


print()
print()


query_2014 = """SELECT
    t.teamID,
    t.yearID,
    t.franchID,
    t.W AS wins,
    t.G AS games_played,
    COALESCE(s.total_payroll, 0) AS total_payroll,
    CASE
        WHEN t.G = 0 THEN NULL
        ELSE (t.W * 1.0 / t.G) * 100.0  -- Avoid integer division
    END AS winning_percentage
FROM
    Teams t
LEFT JOIN (
    SELECT
        teamID,
        yearID,
        SUM(salary) AS total_payroll
    FROM
        Salaries
    GROUP BY
        teamID,
        yearID
) s ON t.teamID = s.teamID AND t.yearID = s.yearID WHERE t.yearID == 2014 """

# only has the data form the 2014 season 
results = conn.execute(query_2014).fetchall()

for rows in results:
    print(rows)

years = pd.read_sql(query, conn)
data_2014 = years[years['yearID'] == 2014]
print(years)

conn.close()

OperationalError: no such table: Teams

## Part 2: Exploratory Data Analysis 

# Problem 2

In [ ]:
conn = sqlite3.connect("lahman2014.sqlite")

query_payrolls = """
SELECT
    teamID,
    yearID,
    sum(salary) as total_payroll
FROM
    Salaries
WHERE
    yearID BETWEEN 1990 AND 2014
GROUP BY
    teamID,
    yearID
"""

payrolls_data = pd.read_sql(query_payrolls, conn)

conn.close()

payrolls_pivot = payrolls_data.pivot(index='yearID', columns='teamID', values='total_payroll')



plt.figure(figsize=(10, 8))

colors = plt.cm.tab20.colors

for i, team in enumerate(payrolls_pivot.columns):
    plt.plot(payrolls_pivot.index, payrolls_pivot[team], label=team, color=colors[i % len(colors)])

plt.xlabel('Year')
plt.ylabel('Payroll (Millions USD ($))')
plt.title('Payrolls Across Teams 1990-2014')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.grid(True)
plt.tight_layout()
plt.savefig('payrolls_across_teams.png')
plt.show()

DatabaseError: Execution failed on sql '
SELECT
    teamID,
    yearID,
    sum(salary) as total_payroll
FROM
    Salaries
WHERE
    yearID BETWEEN 1990 AND 2014
GROUP BY
    teamID,
    yearID
': no such table: Salaries

#### Question 1

What statements can you make about the distribution of payrolls conditioned on time based on these plots? 
Remember you can make statements in terms of central tendency, spread, etc.



After pliotting the payrol over all the teams between 1990 and 2014, You can make the statements that the payroll has been on an upward trend, but has taken a more drastic increase in the more redcent years. Or that there is only one outlier in the data set which is the ATL Braves

#### Problem 3
Write code to produce plots that specifically show at least one of the statements you made in Question 1. For example, if you make a statement that there is a trend for payrolls to decrease over time, make a plot of a statistic for central tendency (e.g., mean payroll) vs. time to show that specifically.

In [ ]:
conn = sqlite3.connect("lahman2014.sqlite")

query_salaryAVG = """
SELECT 
    yearID,
    AVG(salary) AS avg_salary
    FROM
        Salaries
    WHERE
        yearID BETWEEN 1990 AND 2014
    GROUP BY
        yearID
     """

payrolls_data = pd.read_sql(query_salaryAVG, conn)

conn.close()

plt.figure(figsize=(10, 8))

avg_salary = []
for i in range(len(payrolls_data)):
    avg_salary.append(payrolls_data["avg_salary"][i])

plt.plot(payrolls_data["yearID"], avg_salary, linestyle='-', marker='o', markersize=4, linewidth=2, color='blue')

plt.xlabel('Year')
plt.ylabel('Average Salary (All Teams)')
plt.title('Average Salary by Year')
plt.grid(True)
plt.legend(['Average Salary'], loc='upper left', bbox_to_anchor=(1, 1))  # Adjust legend
plt.tight_layout()
plt.savefig('salary_by_year.png')



plt.tight_layout()

plt.show()
plt.close()



DatabaseError: Execution failed on sql '
SELECT 
    yearID,
    AVG(salary) AS avg_salary
    FROM
        Salaries
    WHERE
        yearID BETWEEN 1990 AND 2014
    GROUP BY
        yearID
     ': no such table: Salaries

### Correlation between payroll and winning percentage
#### Problem 4


OperationalError: near "LEFT": syntax error